### Sorting and visualizing UniPept lowest common ancestor analyses in Trocas7

#### Beginning with: LCA peptides in csvs

#### Goal: spectral abundance-corrected (NAAF) taxonomic peptide compositions at the 4 stations before and after 24hrs

## Issue: the `join` commands at the end for diatom and bacterial peptides need to be run independently from one another. To do this, run the diatom command, restart the kernal, then run the bacterial. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
cd /home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept/

/home/millieginty/Documents/git-repos/amazon/analyses/T7-incubations/unipept


In [4]:
# read the data into pandas dataframes
MS_T00_GF_110a = pd.read_csv("lca/cleaned/110_MS_T00_GF_lca.csv")

# delete any Metazoa hits because they're trypsin
MS_T00_GF_110b = MS_T00_GF_110a[MS_T00_GF_110a.kingdom != 'Metazoa']

# keep only entries to the phylum level
MS_T00_GF_110 = MS_T00_GF_110b[MS_T00_GF_110b['phylum'].notnull()]

# How many peptides to the phylum level?
print('# of phylum peptides = ', len(MS_T00_GF_110))

MS_T00_GF_110.head(6)

# of phylum peptides =  24


,peptide,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,...,tribe,subtribe,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma
2,VVEVSLPR,Planctomycetales,Bacteria,NaN,NaN,NaN,Planctomycetes,NaN,NaN,Planctomycetia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,VATVSPLR,Solirubrobacteraceae,Bacteria,NaN,NaN,NaN,Actinobacteria,NaN,NaN,Thermoleophilia,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,LATVLSPR,Gammaproteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,APVLSMR,Brevundimonas,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Alphaproteobacteria,...,Brevundimonas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,EFHPGLAR,Oleiphilus messinensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,...,Oleiphilus,NaN,NaN,NaN,Oleiphilus messinensis,NaN,NaN,NaN,NaN,NaN
48,FFLTGWR,Comamonadaceae,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Betaproteobacteria,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Now I want to read in the file containing the stripped peptides with NAAF values
####  - NAAF stands for 'noramlized area abunace factor'

### I want to join the dataframes if they share an index (stripped peptide with equated leucine and isoleucines)
#### - That means I'll reindex the processed peptide file

In [8]:
MS_T00_GF_110_NAAFa = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/110_MS_T00_GF_DN50_ILnaafs.csv")
MS_T00_GF_110_NAAFb = pd.read_csv("/home/millieginty/Documents/git-repos/amazon/data/Trocas7-incubations/processed/I-L_NAAFs/110B_MS_T00_GF_DN50_ILnaafs.csv")

frames = [MS_T00_GF_110_NAAFa, MS_T00_GF_110_NAAFb]

MS_T00_GF_110_NAAF = pd.concat(frames, sort=False)

MS_T00_GF_110_NAAF.set_index('stripped_peptide')
MS_T00_GF_110_NAAF = MS_T00_GF_110_NAAF.loc[:, ~MS_T00_GF_110_NAAF.columns.str.contains('^Unnamed')]

MS_T00_GF_110_NAAF.rename(columns = {'stripped_peptide':'peptide'}, inplace = True)

print('# of total peptides = ', len(MS_T00_GF_110_NAAF))

print('column names:', MS_T00_GF_110_NAAF.columns)

MS_T00_GF_110_NAAF.head()

# of total peptides =  602
column names: Index(['peptide', 'Area', 'NAAF_num.'], dtype='object')


,peptide,Area,NAAF_num.
0,LSSPATLNSR,22900000.0,2290000.0
1,LSSPATLNSR,22900000.0,2290000.0
2,LSSPATLNSR,22900000.0,2290000.0
3,LSSPATLDSR,190000.0,19000.0
4,VVEVSLPR,652000.0,81500.0


In [10]:
# get the intersection of the phylum peptides and corresponding peptides w/ NAAFs

over = np.intersect1d(MS_T00_GF_110["peptide"], MS_T00_GF_110_NAAF["peptide"])

print(over)

['APVLSMR' 'EFHPGLAR' 'FFLTGWR' 'LAAVSTHK' 'LATVLSPR' 'LATVNVPR'
 'LNYDYAPR' 'QAAWPLAR' 'TGPVAEPR' 'VATVSPLR' 'VVEVSLPR']


In [11]:
# join the dataframes if the peptide values are the same using 'join'
# since a couple are de novo only (more for bacteria), we won't have all the UniPept peptides overlap 

MS_T00_GF_110.set_index('peptide', inplace=True)
MS_T00_GF_110_NAAF.set_index('peptide', inplace=True)

MS_T00_GF_110_Phy = MS_T00_GF_110.join(MS_T00_GF_110_NAAF, how='left', rsuffix='_other')


print('# of total phylum-level peptides = ', len(MS_T00_GF_110_Phy))

MS_T00_GF_110_Phy.head()

# of total phylum-level peptides =  40


,lca,superkingdom,kingdom,subkingdom,superphylum,phylum,subphylum,superclass,class,subclass,...,genus,subgenus,species group,species subgroup,species,subspecies,varietas,forma,Area,NAAF_num.
peptide,,,,,,,,,,,,,,,,,,,,,
ADVLPVGK,Lachnospiraceae,Bacteria,NaN,NaN,NaN,Firmicutes,NaN,NaN,Clostridia,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALLEAAGVPLAAPSANR,Proteobacteria,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ALTVVDPR,Opitutae,Bacteria,NaN,NaN,NaN,Verrucomicrobia,NaN,NaN,Opitutae,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
APVLSMR,Brevundimonas,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Alphaproteobacteria,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10800.0,1542.857143
EFHPGLAR,Oleiphilus messinensis,Bacteria,NaN,NaN,NaN,Proteobacteria,NaN,NaN,Gammaproteobacteria,NaN,...,NaN,NaN,Oleiphilus messinensis,NaN,NaN,NaN,NaN,NaN,471000.0,58875.000000


In [12]:
# write to a csv

MS_T00_GF_110_Phy.to_csv("lca/NAAF/MS_T00_GF_110_Phy_naaf.csv")